In [1]:
# setx JAVA_HOME "C:\Program Files\Java\jdk-23"
# setx PATH "%PATH%;%JAVA_HOME%\bin;%JAVA_HOME%\bin\server"

import jpype
import pdfplumber as pdfl
from tabula import read_pdf
import pandas as pd
import numpy as np

pdf_path = "REPORTE DE GACETA SEMANA DEL 17 AL 21 DE FEBRERO.pdf"
#Obtener el numero de paginas del pdf
with pdfl.open(pdf_path) as pdf:
    num_paginas = len(pdf.pages)

# Lista para acumular los DataFrames de cada página
dfs = []

# Iterar sobre todas las páginas del PDF
for hoja in range(1, num_paginas + 1):
    # Definir el área de extracción (diferente para la primera página)
    area = [100, 10, 800, 1000] if hoja == 1 else [10, 10, 800, 1000]

    # Extraer la tabla de la página actual
    df_pagina = read_pdf(pdf_path, pages=str(hoja), force_subprocess=True, encoding="latin1", area=area, stream=True)

    # Si `read_pdf()` devuelve una lista de DataFrames (varias tablas en una página), las combinamos
    if isinstance(df_pagina, list):
        df_pagina = pd.concat(df_pagina, ignore_index=True)


    # Si es la primera hoja, guardamos el encabezado original
    if hoja == 1:
        df_pagina = df_pagina.drop(df_pagina.columns[5],axis=1)
        encabezado_original = df_pagina.columns.tolist()  # Guardamos el encabezado real
        dfAcumulador = df_pagina  # Guardamos la tabla completa
    else:
        # Guardar el encabezado detectado en la página como una nueva fila
        df_encabezado_pagina = pd.DataFrame([df_pagina.columns.tolist()], columns=df_pagina.columns)
        # Concatenar encabezado de la página como fila + los datos de la tabla
        df_pagina = pd.concat([df_encabezado_pagina, df_pagina], ignore_index=True)
        # Asegurar que todas las páginas tengan el mismo número de columnas
        df_pagina.columns = encabezado_original  # Reasignamos el encabezado original
        # Agregar la tabla procesada al acumulador
        dfAcumulador = pd.concat([dfAcumulador, df_pagina], ignore_index=True)

# Filtrar filas que contienen "TOTAL" en cualquier columna
df_filtrado = dfAcumulador[~dfAcumulador.apply(lambda col: col.map(lambda x: isinstance(x, str) and "total" in x.lower())).any(axis=1)]
df_filtrado.replace(to_replace=r"Unnamed.*", value=np.nan, regex=True, inplace=True)
# Obtener los nombres de las dos primeras columnas dinámicamente
col_0, col_1 = df_filtrado.columns[:2]
# Identificar las filas desalineadas sin necesidad de conocer los nombres de las columnas
mask_desalineadas = (
    df_filtrado[col_0].isna() | 
    df_filtrado[col_1].isna() 
)

# Fusionar solo los valores de las filas desalineadas con la fila anterior en la misma columna
for i in range(1, len(df_filtrado)):
    if mask_desalineadas.iloc[i]:  # Si la fila está desalineada
        for col in df_filtrado.columns:  # Iteramos por cada columna
            if pd.notna(df_filtrado.iloc[i, df_filtrado.columns.get_loc(col)]):  # Si hay datos en la celda
                df_filtrado.iloc[i - 1, df_filtrado.columns.get_loc(col)] += " " + df_filtrado.iloc[i, df_filtrado.columns.get_loc(col)]  # Fusionar con la anterior

# Eliminar las filas desalineadas después de fusionarlas
df_filtrado = df_filtrado[~mask_desalineadas].reset_index(drop=True)
# Guardar el DataFrame corregido en CSV
df_filtrado.to_csv("datos_extraidos.csv", index=False, sep=";")
df_filtrado

Got stderr: mar 18, 2025 7:58:54 A. M. org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

C:\Users\venic\AppData\Local\Temp\ipykernel_38420\4267725112.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.replace(to_replace=r"Unnamed.*", value=np.nan, regex=True, inplace=True)


,Gaceta,Publicación,File Number,Denominación del Signo,Clase,Solicitante,Origen
0,Gazette,Publication,Expediente No,Distinctive Sign,Class,Applicant,Origin
1,31,18FEB25,2024-004018,MIRRINGO,31,"MIZOOCO, S.A.S.",COLOMBIA
2,31,18FEB25,2025-000028,REFLUVIA,5,JDS CARE INTERNATIONAL INC.,PANAMA
3,31,18FEB25,2024-003939,MI YOTA,14,CITIZEN TOKEI KABUSHIKI KAISHA ALSO TRADING AS...,JAPON
4,31,18FEB25,2025-000032,SPECIAL DESIGN,30,"COOPERATIVA DE PRODUCTORES DE LECHE DOS PINOS,...",COSTA RICA
...,...,...,...,...,...,...,...
117,33,19FEB25,2024-003522,BOLYCHIC,30,ROGER ANTONIO GUEVARA RUIZ,NICARAGUA
118,33,19FEB25,2024-003987,SPECIAL DESIGN,5,NOVARTIS AG,SUIZA
119,33,19FEB25,2024-003967,NAMUTEK,"9, 36, 42","NAMUTEK, SOCIEDAD ANONIMA",GUATEMALA
120,33,19FEB25,2024-003241,MANDÁ LOS RIALES,S/P,"AIRPAK INTERNATIONAL (NICARAGUA), SOCIEDAD ANO...",NICARAGUA
